In [19]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [20]:
X_balance_scale = pd.read_csv('data/X_balance_scale', delimiter=',')
y_balance_scale = pd.read_csv('data/y_balance_Scale', delimiter=',')

X_car_eval = pd.read_csv('data/X_car_eval', delimiter=',')
y_car_eval = pd.read_csv('data/y_car_eval', delimiter=',')

X_contraceptive = pd.read_csv('data/X_contraceptive', delimiter=',')
y_contraceptive = pd.read_csv('data/y_contraceptive', delimiter=',')

X_dermatology = pd.read_csv('data/X_dermatology', delimiter=',')
y_dermatology = pd.read_csv('data/y_dermatology', delimiter=',')

X_glass = pd.read_csv('data/X_glass', delimiter=',')
y_glass = pd.read_csv('data/y_glass', delimiter=',')

X_hayes = pd.read_csv('data/X_hayes', delimiter=',')
y_hayes = pd.read_csv('data/y_hayes', delimiter=',')

X_heart = pd.read_csv('data/X_heart', delimiter=',')
y_heart = pd.read_csv('data/y_heart', delimiter=',')

X_new_thyroid = pd.read_csv('data/X_new_thyroid', delimiter=',')
y_new_thyroid = pd.read_csv('data/y_new_thyroid', delimiter=',')

X_page = pd.read_csv('data/X_page', delimiter=',')
y_page = pd.read_csv('data/y_page', delimiter=',')

X_pen_based = pd.read_csv('data/X_pen_based', delimiter=',')
y_pen_based = pd.read_csv('data/y_pen_based', delimiter=',')

X_shuttle = pd.read_csv('data/X_shuttle', delimiter=',')
y_shuttle = pd.read_csv('data/y_shuttle', delimiter=',')

X_vertebra = pd.read_csv('data/X_vertebra', delimiter=',')
y_vertebra = pd.read_csv('data/y_vertebra', delimiter=',')

X_wine = pd.read_csv('data/X_wine', delimiter=',')
y_wine = pd.read_csv('data/y_wine', delimiter=',')

X_yeast = pd.read_csv('data/X_yeast', delimiter=',')
y_yeast = pd.read_csv('data/y_yeast', delimiter=',')

X_fars = pd.read_csv('data/X_fars', delimiter=',')
y_fars = pd.read_csv('data/y_fars', delimiter=',')

In [21]:
X_list = [
    X_balance_scale, X_car_eval, X_contraceptive,
    X_dermatology, X_glass, X_hayes, X_heart,
    X_new_thyroid, X_page, X_pen_based, X_shuttle,
    X_vertebra, X_wine, X_yeast, X_fars
]

y_list = [
    y_balance_scale, y_car_eval, y_contraceptive,
    y_dermatology, y_glass, y_hayes, y_heart,
    y_new_thyroid, y_page, y_pen_based, y_shuttle,
    y_vertebra, y_wine, y_yeast, y_fars
]

name_list = [
    'Balance Scale', 'Car Evaluation', 'Contraceptive',
    'Dermatology', 'Glass', 'Hayes', 'Heart',
    'New Thyroid', 'Page', 'Pen Based', 'Shuttle',
    'Vertebra', 'Wine', 'Yeast', 'FARS'
]

# Prediction

Using 7 boosting approaches to predict initial data

In [40]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.ensemble import RUSBoostClassifier
from adacost import AdaCost
from utils import *
from tqdm import tqdm

In [23]:
clf_dict = {
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42),
    'AdaBoost_SAMME': AdaBoostClassifier(n_estimators=100, algorithm='SAMME', random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.3, max_features=2, max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'CatBoost': CatBoostClassifier(verbose=0), 
    'RUSBoostClassifier': RUSBoostClassifier(n_estimators=200, random_state=42),
    'AdaCost' : AdaCost(algorithm = "SAMME.R",random_state = 100)
}

clf_score_dict = {}

In [25]:
for clf_name, clf in tqdm(clf_dict.items()):
    score = {}
    
    for i, (X, y) in enumerate(zip(X_list, y_list)):

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        clf.fit(X_train, np.ravel(y_train)) 
        
        y_preds = clf.predict(X_test) 
        y_scores = clf.predict_proba(X_test)

        score_p = metric_list(y_test,y_preds,y_scores, len(y.iloc[:,0].unique()))
        score[name_list[i]]= score_p
    
    df = pd.DataFrame.from_dict(score, orient='index', columns=['accuracy', 'precision', 'recall', 'f1','Macro-Averaged AUPRC',
                                                            'F_measure Beta=10','Modified mcc','MCC metirc','Gmean'])
    clf_score_dict[clf_name] = df
    
    #print(clf_score_dict[clf_name])

100%|██████████| 7/7 [04:03<00:00, 34.80s/it]


In [32]:
clf_score_dict['AdaBoost']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.944000,0.965778,0.944000,0.949436,0.697356,0.945939,0.882886,0.910854,0.881972
Car Evaluation,0.832370,0.833482,0.832370,0.821514,0.685982,0.832471,0.667945,0.643877,0.691205
Contraceptive,0.589831,0.596148,0.589831,0.585984,0.502358,0.590399,0.358602,0.365253,0.233743
Dermatology,0.716216,0.619248,0.716216,0.645555,0.643528,0.706164,0.578996,0.666142,0.000000
Glass,0.488372,0.481190,0.488372,0.425292,0.583236,0.487710,0.341356,0.326144,0.000000
Hayes,0.500000,0.307692,0.500000,0.361111,0.833333,0.473118,0.441292,0.486427,1.000000
Heart,0.524590,0.446643,0.524590,0.474243,0.322913,0.516397,0.260505,0.286965,0.000000
New Thyroid,0.720930,0.618605,0.720930,0.633926,0.653704,0.710250,0.331742,0.401677,0.000000
Page,0.738813,0.915992,0.738813,0.794022,0.676391,0.752037,0.517763,0.421373,0.000000
Pen Based,0.283765,0.243578,0.283765,0.170141,0.199340,0.279572,0.212055,0.256988,0.000000


In [33]:
clf_score_dict['AdaBoost_SAMME']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.888000,0.857166,0.888000,0.868756,0.684327,0.885106,0.621345,0.805109,0.932203
Car Evaluation,0.849711,0.868063,0.849711,0.840497,0.545536,0.851347,0.594444,0.685234,0.040596
Contraceptive,0.576271,0.609650,0.576271,0.573188,0.614151,0.579154,0.351266,0.350759,0.213665
Dermatology,0.878378,0.905405,0.878378,0.868903,0.843493,0.880769,0.837876,0.848979,0.200000
Glass,0.627907,0.613999,0.627907,0.613924,0.602452,0.626617,0.499431,0.513841,0.000000
Hayes,0.500000,0.307692,0.500000,0.361111,0.833333,0.473118,0.441292,0.486427,1.000000
Heart,0.540984,0.481981,0.540984,0.507148,0.348865,0.535029,0.298713,0.316025,0.000000
New Thyroid,0.976744,0.977546,0.976744,0.976054,0.981400,0.976817,0.954171,0.955106,0.857143
Page,0.949772,0.947905,0.949772,0.948765,0.695365,0.949602,0.719292,0.737964,0.064709
Pen Based,0.676671,0.674940,0.676671,0.667727,0.666311,0.676513,0.640880,0.642441,0.010803


In [35]:
clf_score_dict['GradientBoosting']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.816000,0.787066,0.816000,0.798012,0.697514,0.813282,0.492666,0.679347,8.003082e-01
Car Evaluation,0.965318,0.975643,0.965318,0.968380,0.960514,0.966247,0.914174,0.928990,7.486631e-01
Contraceptive,0.535593,0.539368,0.535593,0.533688,0.574399,0.535934,0.273517,0.280102,1.833983e-01
Dermatology,0.986486,0.987838,0.986486,0.986392,1.000000,0.986609,0.980653,0.982152,8.750000e-01
Glass,0.837209,0.856848,0.837209,0.829605,0.982895,0.838957,0.801831,0.795590,2.380952e-01
Hayes,0.781250,0.829545,0.781250,0.782870,0.951577,0.785407,0.724931,0.687619,9.000000e-01
Heart,0.491803,0.379372,0.491803,0.425061,0.375161,0.478901,0.191558,0.204533,0.000000e+00
New Thyroid,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00
Page,0.969863,0.969911,0.969863,0.969765,0.776024,0.969867,0.821565,0.845914,3.442390e-01
Pen Based,0.994088,0.994114,0.994088,0.994088,0.999565,0.994091,0.993442,0.993432,9.461875e-01


In [36]:
clf_score_dict['XGBoost']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.840000,0.829746,0.840000,0.833185,0.715320,0.839057,0.558187,0.722230,0.832974
Car Evaluation,0.979769,0.987636,0.979769,0.981765,0.943689,0.980479,0.945153,0.958866,1.000000
Contraceptive,0.586441,0.588074,0.586441,0.584939,0.582482,0.586589,0.353759,0.358250,0.247228
Dermatology,0.986486,0.987838,0.986486,0.986392,1.000000,0.986609,0.980653,0.982152,0.875000
Glass,0.837209,0.840091,0.837209,0.834931,0.911404,0.837470,0.786555,0.790912,0.261905
Hayes,0.812500,0.832418,0.812500,0.815020,0.952469,0.814271,0.734514,0.707742,0.750000
Heart,0.475410,0.407401,0.475410,0.438764,0.329632,0.468303,0.201437,0.218317,0.000000
New Thyroid,0.976744,0.977546,0.976744,0.976054,0.999582,0.976817,0.954171,0.955106,0.857143
Page,0.971689,0.972437,0.971689,0.971877,0.832299,0.971757,0.835902,0.858275,0.409673
Pen Based,0.990450,0.990513,0.990450,0.990450,0.999304,0.990456,0.989422,0.989393,0.911706


In [37]:
clf_score_dict['CatBoost']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.880000,0.846267,0.880000,0.860520,0.723314,0.876823,0.608943,0.788488,0.915254
Car Evaluation,0.973988,0.980708,0.973988,0.975628,0.978974,0.974596,0.933166,0.946918,0.909091
Contraceptive,0.549153,0.547273,0.549153,0.543968,0.564997,0.548981,0.289413,0.297712,0.182649
Dermatology,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Glass,0.906977,0.914341,0.906977,0.907780,0.970088,0.907641,0.883432,0.882985,0.589286
Hayes,0.718750,0.767857,0.718750,0.716667,0.912292,0.722953,0.585374,0.561444,0.450000
Heart,0.475410,0.381148,0.475410,0.423059,0.354687,0.464956,0.181516,0.205881,0.000000
New Thyroid,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Page,0.975342,0.976794,0.975342,0.975681,0.887257,0.975474,0.858844,0.877837,0.623546
Pen Based,0.994543,0.994578,0.994543,0.994543,0.999725,0.994546,0.993937,0.993939,0.950758


In [38]:
clf_score_dict['RUSBoostClassifier']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.912000,0.914672,0.912000,0.912281,0.716417,0.912242,0.793152,0.848966,0.881972
Car Evaluation,0.846821,0.848332,0.846821,0.844034,0.703941,0.846958,0.688469,0.676854,0.497303
Contraceptive,0.444068,0.523261,0.444068,0.454830,0.443577,0.450263,0.199300,0.203306,0.173959
Dermatology,0.891892,0.834658,0.891892,0.854470,0.991461,0.886366,0.854084,0.868796,0.000000
Glass,0.790698,0.809561,0.790698,0.783187,0.851718,0.792376,0.729673,0.729528,0.142857
Hayes,0.687500,0.495192,0.687500,0.568452,0.833333,0.664056,0.493461,0.567498,0.000000
Heart,0.524590,0.493429,0.524590,0.506668,0.352792,0.521596,0.290420,0.312659,0.000000
New Thyroid,0.953488,0.956072,0.953488,0.953278,0.975967,0.953723,0.913362,0.910528,0.857143
Page,0.887671,0.953205,0.887671,0.910667,0.678908,0.893254,0.646440,0.651354,0.472973
Pen Based,0.293770,0.284901,0.293770,0.215929,0.252147,0.292941,0.227481,0.246286,0.000000


In [39]:
clf_score_dict['AdaCost']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale,0.560000,0.514409,0.560000,0.531522,0.568625,0.555524,0.144254,0.200678,0.372573
Car Evaluation,0.679191,0.461300,0.679191,0.549431,0.661212,0.651227,0.000000,0.000000,0.000000
Contraceptive,0.440678,0.194197,0.440678,0.269591,0.692132,0.395091,0.000000,0.000000,0.000000
Dermatology,0.594595,0.460204,0.594595,0.495316,0.656855,0.579218,0.353353,0.518013,0.000000
Glass,0.511628,0.316279,0.511628,0.372093,0.666667,0.484427,0.311927,0.437439,0.000000
Hayes,0.343750,0.288306,0.343750,0.206010,0.693604,0.337844,0.164989,0.181865,0.166667
Heart,0.475410,0.226015,0.475410,0.306375,0.591959,0.432068,0.000000,0.000000,0.000000
New Thyroid,0.790698,0.655563,0.790698,0.711270,0.808895,0.776153,0.487803,0.590638,0.000000
Page,0.930594,0.897045,0.930594,0.913246,0.658874,0.927440,0.405152,0.605600,0.000000
Pen Based,0.203274,0.061138,0.203274,0.088825,0.560006,0.167808,0.106059,0.179551,0.000000


In [41]:
file_name_list = []
def save_result(result,file_name_list):
    for clf, score in result.items():
        score.to_csv(f'result/models/{clf}.csv', index=True)
        file_name_list.append(f'{clf}.csv')
save_result(clf_score_dict,file_name_list)